In [110]:
# import nesseccary modules
import requests as r
import pandas as pd
import json as j
import sql_functions as sf
import psycopg2
import os
key = os.getenv('meteostat_api_key') # you need your own key in the .env file with the according keyword



In [ ]:
#set url and parameters to the ones specified on https://rapidapi.com/meteostat/api/meteostat/

url = "https://meteostat.p.rapidapi.com/stations/hourly"
querystring = {"station": "14839", "start": "2020-01-01", "end": "2020-01-31"}
headers = {
    "X-RapidAPI-Key": key,
    "X-RapidAPI-Host": "meteostat.p.rapidapi.com"
}

r_mke = r.get(url, headers=headers, params=querystring)

print(r_mke.json())

okay there seems to be no data. check for another station

In [11]:

querystring = {"station": "72550", "start": "2020-01-01", "end": "2020-01-31"}

r_oma = r.get(url, headers=headers, params=querystring)
print(r_oma.json())

{'meta': {'generated': '2024-01-21 23:40:21'}, 'data': [{'time': '2020-01-01 00:00:00', 'temp': 1.1, 'dwpt': -7.2, 'rhum': 54.0, 'prcp': None, 'snow': None, 'wdir': 0.0, 'wspd': 0.0, 'wpgt': None, 'pres': 1014.5, 'tsun': None, 'coco': None}, {'time': '2020-01-01 01:00:00', 'temp': -1.7, 'dwpt': -6.6, 'rhum': 69.0, 'prcp': 0.0, 'snow': None, 'wdir': 180.0, 'wspd': 5.4, 'wpgt': None, 'pres': 1013.9, 'tsun': None, 'coco': None}, {'time': '2020-01-01 02:00:00', 'temp': -2.8, 'dwpt': -6.1, 'rhum': 78.0, 'prcp': 0.0, 'snow': None, 'wdir': 150.0, 'wspd': 7.6, 'wpgt': None, 'pres': 1013.7, 'tsun': None, 'coco': None}, {'time': '2020-01-01 03:00:00', 'temp': -2.2, 'dwpt': -6.0, 'rhum': 75.0, 'prcp': 0.0, 'snow': None, 'wdir': 160.0, 'wspd': 7.6, 'wpgt': None, 'pres': 1013.2, 'tsun': None, 'coco': None}, {'time': '2020-01-01 04:00:00', 'temp': -3.9, 'dwpt': -6.7, 'rhum': 81.0, 'prcp': 0.0, 'snow': None, 'wdir': 160.0, 'wspd': 11.2, 'wpgt': None, 'pres': 1012.8, 'tsun': None, 'coco': None}, {'tim

success! seems the weather station mke has no data (from that time). continue coding with OMA at frist

In [14]:
oma_w = r_oma.json() #save as json object
#but what is the form, is there need for flattening?

# Create json object: json_object
json_object = j.loads(r_oma.content)

print(j.dumps(json_object, indent = 3))

{
   "meta": {
      "generated": "2024-01-21 23:40:21"
   },
   "data": [
      {
         "time": "2020-01-01 00:00:00",
         "temp": 1.1,
         "dwpt": -7.2,
         "rhum": 54.0,
         "prcp": null,
         "snow": null,
         "wdir": 0.0,
         "wspd": 0.0,
         "wpgt": null,
         "pres": 1014.5,
         "tsun": null,
         "coco": null
      },
      {
         "time": "2020-01-01 01:00:00",
         "temp": -1.7,
         "dwpt": -6.6,
         "rhum": 69.0,
         "prcp": 0.0,
         "snow": null,
         "wdir": 180.0,
         "wspd": 5.4,
         "wpgt": null,
         "pres": 1013.9,
         "tsun": null,
         "coco": null
      },
      {
         "time": "2020-01-01 02:00:00",
         "temp": -2.8,
         "dwpt": -6.1,
         "rhum": 78.0,
         "prcp": 0.0,
         "snow": null,
         "wdir": 150.0,
         "wspd": 7.6,
         "wpgt": null,
         "pres": 1013.7,
         "tsun": null,
         "coco": null
      

in data, a list is given, but we are not intersted in meta anyway

In [18]:
#lets see what the flattening gives us
oma_w_n = pd.json_normalize(oma_w['data'], sep="_") # important to just take data!
oma_w_n

,time,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
0,2020-01-01 00:00:00,1.1,-7.2,54.0,NaN,None,0.0,0.0,None,1014.5,None,NaN
1,2020-01-01 01:00:00,-1.7,-6.6,69.0,0.0,None,180.0,5.4,None,1013.9,None,NaN
2,2020-01-01 02:00:00,-2.8,-6.1,78.0,0.0,None,150.0,7.6,None,1013.7,None,NaN
3,2020-01-01 03:00:00,-2.2,-6.0,75.0,0.0,None,160.0,7.6,None,1013.2,None,NaN
4,2020-01-01 04:00:00,-3.9,-6.7,81.0,0.0,None,160.0,11.2,None,1012.8,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
739,2020-01-31 19:00:00,1.7,-0.6,85.0,NaN,None,330.0,16.6,None,1016.3,None,NaN
740,2020-01-31 20:00:00,1.7,-0.6,85.0,0.0,None,330.0,20.5,None,1016.0,None,NaN
741,2020-01-31 21:00:00,1.7,-0.6,85.0,0.0,None,330.0,16.6,None,1016.5,None,NaN
742,2020-01-31 22:00:00,1.7,-0.6,85.0,0.0,None,300.0,13.0,None,1016.8,None,NaN


In [19]:
#to be able to stack the different df together an identifying clolum needs to be added-> airportcode

oma_w_n['faa'] = 'OMA'
oma_w_n.head()

,time,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco,faa
0,2020-01-01 00:00:00,1.1,-7.2,54.0,NaN,None,0.0,0.0,None,1014.5,None,NaN,OMA
1,2020-01-01 01:00:00,-1.7,-6.6,69.0,0.0,None,180.0,5.4,None,1013.9,None,NaN,OMA
2,2020-01-01 02:00:00,-2.8,-6.1,78.0,0.0,None,150.0,7.6,None,1013.7,None,NaN,OMA
3,2020-01-01 03:00:00,-2.2,-6.0,75.0,0.0,None,160.0,7.6,None,1013.2,None,NaN,OMA
4,2020-01-01 04:00:00,-3.9,-6.7,81.0,0.0,None,160.0,11.2,None,1012.8,None,NaN,OMA


In [75]:
#since the waetherstation codes are inconsistent, back to lat and lon
#first a list with the aiport codes
s = 'hh_analytics_23_4'
faa_list = ['MKE','ORD','IND','OMA','DSM','STL']

# now the lat and lon lists will be filled with the according values from the faa_list with a query in a for loop
lat_list = []
lon_list = []
for i in faa_list:
    new_lat = sf.get_dataframe(f"select distinct lat from {s}.airports where faa LIKE '{i}';").to_string()
    new_lat_f = float(new_lat.replace('lat\n0',''))
    lat_list.append(new_lat_f)
    new_lon = sf.get_dataframe(f"select distinct lon from {s}.airports where faa like '{i}';").to_string()
    new_lon_f = float(new_lon.replace('lon\n0',''))
    lon_list.append(new_lon_f)

print(lat_list)
print(lon_list)
# learning, in queries strings HAVE to be put in '' not "" there for the "" has to be on the outside of the query
# find for a nicer way to just get the values of the result in a list

[42.947201, 41.9786, 39.7173, 41.3032, 41.534, 38.748697]
[-87.896599, -87.9048, -86.294403, -95.894096, -93.663101, -90.370003]


In [39]:
#now we need to update on the API, to avoid mix ups or unnesserary calls, varables are renamed

url_l = "https://meteostat.p.rapidapi.com/point/hourly"

querystring_l = {"lat":"43.6667","lon":"-79.4","start":"2020-01-01","end":"2020-01-01","alt":"113","tz":"America/Toronto"}

headers = {
	"X-RapidAPI-Key": "120bf7b5cemsh908715b0d7573a8p124218jsn672a07501c61",
	"X-RapidAPI-Host": "meteostat.p.rapidapi.com"
}#stays the same, kept here for übersichtlichkeit

response = r.get(url_l, headers=headers, params=querystring_l)

# test run
json_object = j.loads(response.content)

print(j.dumps(json_object, indent = 3))

#data looks the same so same method as above can be used or can it?

{
   "meta": {
      "generated": "2024-01-22 00:41:21",
      "stations": [
         "71508",
         "71265",
         "2XUGG",
         "71624"
      ]
   },
   "data": [
      {
         "time": "2020-01-01 00:00:00",
         "temp": -0.1,
         "dwpt": -5.1,
         "rhum": 69.0,
         "prcp": 0.0,
         "snow": null,
         "wdir": 250.0,
         "wspd": 35.3,
         "wpgt": null,
         "pres": 1002.1,
         "tsun": null,
         "coco": 3
      },
      {
         "time": "2020-01-01 01:00:00",
         "temp": -0.1,
         "dwpt": -4.9,
         "rhum": 70.0,
         "prcp": 0.0,
         "snow": null,
         "wdir": 0.0,
         "wspd": 0.0,
         "wpgt": null,
         "pres": 1002.3,
         "tsun": null,
         "coco": 3
      },
      {
         "time": "2020-01-01 02:00:00",
         "temp": -0.1,
         "dwpt": -5.5,
         "rhum": 67.0,
         "prcp": 0.0,
         "snow": null,
         "wdir": 250.0,
         "wspd": 33.5,
   

In [98]:
# test run for the url
for i in range(6):                      # range of indexes needed to go through the lists
    print({"lat":lat_list[i],"lon":lat_list[i],"start":"2020-08-01","end":"2020-08-31"} )

{'lat': 42.947201, 'lon': 42.947201, 'start': '2020-08-01', 'end': '2020-08-31'}
{'lat': 41.9786, 'lon': 41.9786, 'start': '2020-08-01', 'end': '2020-08-31'}
{'lat': 39.7173, 'lon': 39.7173, 'start': '2020-08-01', 'end': '2020-08-31'}
{'lat': 41.3032, 'lon': 41.3032, 'start': '2020-08-01', 'end': '2020-08-31'}
{'lat': 41.534, 'lon': 41.534, 'start': '2020-08-01', 'end': '2020-08-31'}
{'lat': 38.748697, 'lon': 38.748697, 'start': '2020-08-01', 'end': '2020-08-31'}


In [96]:
#to get everything together we use another for loop
import time
city_weather = pd.DataFrame()

for i in range(6):                      # range of indexes needed to go through the lists
    querystring_ll = {"lat":lat_list[i],"lon":lat_list[i],"start":"2020-08-01","end":"2020-08-31"} #automatically updated url
    df = r.get(url_l, headers=headers, params=querystring_ll).json()
    df_n = pd.json_normalize(df['data'], sep="_")
    df_n['faa'] = faa_list[i]
    city_weather= pd.concat([city_weather,df_n])
    time.sleep(10)
    print(f'{i}. airport {faa_list[i]} done')

city_weather.head()

0. airport MKE done
1. airport ORD done
2. airport IND done
3. airport OMA done
4. airport DSM done


/var/folders/9j/5rxmqbbd545bhw9vljt6mdp40000gn/T/ipykernel_60509/4206538302.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  city_weather= pd.concat([city_weather,df_n])


5. airport STL done


,faa,time,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
0,ORD,2020-08-01 00:00:00,23.0,22.0,94.0,None,None,210.0,11.2,None,1007.0,None,NaN
1,ORD,2020-08-01 01:00:00,23.0,22.0,94.0,None,None,130.0,9.4,None,1007.0,None,17.0
2,ORD,2020-08-01 02:00:00,23.0,22.0,94.0,None,None,210.0,7.6,None,1006.0,None,NaN
3,ORD,2020-08-01 03:00:00,23.0,22.0,94.0,None,None,160.0,5.4,None,1006.0,None,NaN
4,ORD,2020-08-01 04:00:00,23.0,22.0,94.0,None,None,130.0,9.4,None,1006.0,None,NaN


In [97]:
city_weather.tail()

,faa,time,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
739,STL,2020-08-31 19:00:00,24.0,1.0,22.0,None,None,260.0,9.4,None,1010.0,None,NaN
740,STL,2020-08-31 20:00:00,24.0,2.8,25.0,None,None,290.0,9.4,None,1010.0,None,NaN
741,STL,2020-08-31 21:00:00,20.5,1.8,29.0,None,None,210.0,5.4,None,1003.4,None,NaN
742,STL,2020-08-31 22:00:00,22.0,2.1,27.0,None,None,310.0,7.6,None,1010.0,None,NaN
743,STL,2020-08-31 23:00:00,18.0,1.9,34.0,None,None,230.0,11.2,None,1010.0,None,NaN


In [109]:
#YAY! now upload to DBeaver  okay no, first, can we drop columns?
 
city_weather.info()
#--> all 0 non-null can be dropped because empty, but what is coco?

<class 'pandas.core.frame.DataFrame'>
Index: 3720 entries, 0 to 743
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   faa     3720 non-null   object 
 1   time    3720 non-null   object 
 2   temp    3717 non-null   float64
 3   dwpt    3717 non-null   float64
 4   rhum    3717 non-null   float64
 5   prcp    0 non-null      object 
 6   snow    0 non-null      object 
 7   wdir    3613 non-null   float64
 8   wspd    3714 non-null   float64
 9   wpgt    0 non-null      object 
 10  pres    3648 non-null   float64
 11  tsun    0 non-null      object 
 12  coco    334 non-null    float64
dtypes: float64(7), object(6)
memory usage: 535.9+ KB



Code	|Weather Condition
-----|----------------------
7	|Light Rain
8	|Rain
17	|Rain Shower
25	|Thunderstorm
[source](https://dev.meteostat.net/formats.html#weather-condition-codes)

In [106]:
city_weather_clean = city_weather.drop(['snow','prcp','wpgt','tsun'], axis=1)
city_weather_clean.head()

,faa,time,temp,dwpt,rhum,wdir,wspd,pres,coco
0,ORD,2020-08-01 00:00:00,23.0,22.0,94.0,210.0,11.2,1007.0,NaN
1,ORD,2020-08-01 01:00:00,23.0,22.0,94.0,130.0,9.4,1007.0,17.0
2,ORD,2020-08-01 02:00:00,23.0,22.0,94.0,210.0,7.6,1006.0,NaN
3,ORD,2020-08-01 03:00:00,23.0,22.0,94.0,160.0,5.4,1006.0,NaN
4,ORD,2020-08-01 04:00:00,23.0,22.0,94.0,130.0,9.4,1006.0,NaN


In [107]:
city_weather_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3720 entries, 0 to 743
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   faa     3720 non-null   object 
 1   time    3720 non-null   object 
 2   temp    3717 non-null   float64
 3   dwpt    3717 non-null   float64
 4   rhum    3717 non-null   float64
 5   wdir    3613 non-null   float64
 6   wspd    3714 non-null   float64
 7   pres    3648 non-null   float64
 8   coco    334 non-null    float64
dtypes: float64(7), object(2)
memory usage: 419.7+ KB


In [113]:
#time for upload
engine= sf.get_engine()

if engine!=None:
    try:
        city_weather_clean.to_sql(name='weather_MW_group04', # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        schema=s, # Use schmea that was defined earlier
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print("The 'weather_MW_group04' table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

The 'weather_MW_group04' table was imported successfully.
